# LMER in rpy2

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from CrossOverExperiment import CrossOverExperiment, RCT

import rpy2

# Install LMER packages (THIS TAKES ABOUT 3~5 minutes)
packnames = ('lme4', 'lmerTest', 'emmeans', "geepack")
from rpy2.robjects.packages import importr, data
from rpy2.robjects.vectors import StrVector

utils = importr("utils")
utils.chooseCRANmirror(ind=1)

import pandas as pd
import rpy2.robjects as robjects
from rpy2.ipython.ggplot import image_png
from rpy2.robjects import pandas2ri# Defining the R script and loading the instance in Python
r = robjects.r

from statsmodels.api import MixedLM


import numpy as np
import pandas as pd

pandas2ri.activate()

### I had to install all these in this order before getting lme4 to work with rpy2

## R imports

In [7]:
lme4 = importr('lme4')
lmertest = importr('lmerTest')
base = importr('base')
stats = importr("stats")

## prepare some test data

In [8]:
crossover_experiment = CrossOverExperiment(subjects=15,periods=6,base_effect=2,mu=1,tau=0.2,carryover=0.1,alpha_sd=2,epsilon_sd=1,error_type="whatever, statsmodels supports only idiosyncratic")
crossover_experiment.params

{'Subjects': 15,
 'Periods': 6,
 'Base Effect': 2,
 'Mu': 1,
 'Tau': 0.2,
 'Carryover Effect': 0.1,
 'Alpha Standard Deviation': 2,
 'Epsilon Standard Deviation': 1,
 'Error Structure': 'whatever, statsmodels supports only idiosyncratic'}

In [9]:
(y0,y1),sub,T,t,cO = crossover_experiment.generate_data(return_for_t=False)

In [10]:
exog = pd.DataFrame(np.array([np.ones(len(T)),T,t,cO,sub]).T)#.reset_index()
exog.columns = ["len_T", "T", "t", "c0", "sub"]

endog = pd.DataFrame(y0)
endog.columns = ["y0"]

endogexog = pd.concat([endog,exog], axis=1)

In [18]:
import statsmodels.api as sm

In [17]:
!pip freeze | grep statsmodels

statsmodels==0.13.2


In [24]:
 mlm0 = sm.MixedLM(endog=y0,exog=np.array([np.ones(len(T)),T,t,cO]).T,groups=sub).fit()#.pvalues[1]

In [25]:
mlm0.pvalues

array([2.00762592e-05, 4.68208609e-01, 7.36077458e-02, 2.90028631e-01,
       2.12449124e-02])

## try 2 ways

In [ ]:
fm1 = lme4.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(base.summary(fm1))

In [26]:
fm2 = lmertest.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(base.summary(fm2))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: "y0 ~ (1|sub) + len_T + T + t + c0"
   Data: 
structure(list(y0 = c(-1.2447649444855, -0.521565199574813, 0.159886812301275,  
0.503712450503257, 0.057632139907297, -0.996068299082466, 2.55613584844934,  
5.55130665986866, 5.5049770966758, 4.96889284945018, 5.04155665368426,  
6.66933794908449, 5.4196055971863, 4.74949422725179, 4.47509810237004,  
6.08825632019904, 4.55750937548632, 6.65558909674113, 3.14541778345784,  
2.35291009461571, 6.56806768932887, 3.21863103793614, 2.84910969392112,  
3.52862254406408, 1.53298222583852, 5.19127586190146, 1.91389881190012,  
2.08135376927231, 2.26271416057902, 0.758822278992021, 1.27715641866361,  
2.22050325208836, 1.26159756146612, 0.744119449773743, 1.66487532596779,  
0.724930022776273, 1.33047454578612, 0.527689317775565, 1.71760323744939,  
0.727724549807137, 2.70767771343855, 2.04486565935174, 1.22250549593351,  
-0.611024821247994, 1.729376876

## try to extract p-values

In [27]:
fm2 = lmertest.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(stats.anova(fm2))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Type III Analysis of Variance Table with Satterthwaite's method
      Sum Sq Mean Sq NumDF  DenDF F value Pr(>F)  
T     0.5491  0.5491     1 72.152  0.5263 0.4705  
t     3.3394  3.3394     1 72.056  3.2009 0.0778 .
c0    1.1684  1.1684     1 72.366  1.1200 0.2934  
len_T                                             
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



In [29]:
fm2anova = stats.anova(fm2)

In [30]:
fm2anova.rx2("Pr(>F)")[0] # p_values

0.4705158304018363